In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

/home/user/anaconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/user/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
env = make_vec_env("G1GoalDist-v1", 12)

/home/user/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/user/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=2048,
    buffer_size=14000000,
    learning_rate=1e-4,
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [5]:
# model.set_parameters("trained_models/g1-reptile/sac")

In [ ]:
model.learn(
    total_timesteps=int(1e7), 
    log_interval=100,
    tb_log_name='reptile',
    reset_num_timesteps=False,
)

Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 117       |
|    ep_rew_mean     | -1.41e+03 |
| time/              |           |
|    episodes        | 100       |
|    fps             | 410       |
|    time_elapsed    | 36        |
|    total_timesteps | 15096     |
| train/             |           |
|    actor_loss      | 68.7      |
|    critic_loss     | 15.2      |
|    ent_coef        | 0.892     |
|    ent_coef_loss   | -1.61     |
|    learning_rate   | 0.0001    |
|    n_updates       | 1249      |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 244       |
|    ep_rew_mean     | -1.58e+03 |
| time/              |           |
|    episodes        | 200       |
|    fps             | 406       |
|    time_elapsed    | 97        |
|    total_timesteps | 39756     |
| train/             |           |
|    actor_loss      | 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 25       |
|    ep_rew_mean     | -132     |
| time/              |          |
|    episodes        | 1600     |
|    fps             | 402      |
|    time_elapsed    | 275      |
|    total_timesteps | 110976   |
| train/             |          |
|    actor_loss      | 143      |
|    critic_loss     | 63.2     |
|    ent_coef        | 0.423    |
|    ent_coef_loss   | -6.05    |
|    learning_rate   | 0.0001   |
|    n_updates       | 9239     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.2     |
|    ep_rew_mean     | -122     |
| time/              |          |
|    episodes        | 1700     |
|    fps             | 402      |
|    time_elapsed    | 281      |
|    total_timesteps | 113364   |
| train/             |          |
|    actor_loss      | 143      |
|    critic_loss     | 69.3     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.9     |
|    ep_rew_mean     | -99.5    |
| time/              |          |
|    episodes        | 3100     |
|    fps             | 401      |
|    time_elapsed    | 361      |
|    total_timesteps | 145068   |
| train/             |          |
|    actor_loss      | 120      |
|    critic_loss     | 48.1     |
|    ent_coef        | 0.329    |
|    ent_coef_loss   | -6.59    |
|    learning_rate   | 0.0001   |
|    n_updates       | 12080    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.3     |
|    ep_rew_mean     | -94.3    |
| time/              |          |
|    episodes        | 3200     |
|    fps             | 401      |
|    time_elapsed    | 366      |
|    total_timesteps | 147240   |
| train/             |          |
|    actor_loss      | 116      |
|    critic_loss     | 48.9     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.4     |
|    ep_rew_mean     | -83.7    |
| time/              |          |
|    episodes        | 4600     |
|    fps             | 401      |
|    time_elapsed    | 439      |
|    total_timesteps | 176160   |
| train/             |          |
|    actor_loss      | 104      |
|    critic_loss     | 39.4     |
|    ent_coef        | 0.261    |
|    ent_coef_loss   | -5.6     |
|    learning_rate   | 0.0001   |
|    n_updates       | 14671    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.3     |
|    ep_rew_mean     | -79.9    |
| time/              |          |
|    episodes        | 4700     |
|    fps             | 401      |
|    time_elapsed    | 444      |
|    total_timesteps | 178200   |
| train/             |          |
|    actor_loss      | 101      |
|    critic_loss     | 35.6     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.8     |
|    ep_rew_mean     | -74.1    |
| time/              |          |
|    episodes        | 6100     |
|    fps             | 400      |
|    time_elapsed    | 514      |
|    total_timesteps | 206268   |
| train/             |          |
|    actor_loss      | 92.5     |
|    critic_loss     | 31       |
|    ent_coef        | 0.212    |
|    ent_coef_loss   | -3.84    |
|    learning_rate   | 0.0001   |
|    n_updates       | 17180    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.4     |
|    ep_rew_mean     | -73.9    |
| time/              |          |
|    episodes        | 6200     |
|    fps             | 400      |
|    time_elapsed    | 519      |
|    total_timesteps | 208188   |
| train/             |          |
|    actor_loss      | 94.9     |
|    critic_loss     | 29.6     |
|    ent_coef 

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.3     |
|    ep_rew_mean     | -29.2    |
| time/              |          |
|    episodes        | 457600   |
|    fps             | 396      |
|    time_elapsed    | 20046    |
|    total_timesteps | 7941936  |
| train/             |          |
|    actor_loss      | 28.1     |
|    critic_loss     | 0.762    |
|    ent_coef        | 0.097    |
|    ent_coef_loss   | -0.166   |
|    learning_rate   | 0.0001   |
|    n_updates       | 661819   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16.6     |
|    ep_rew_mean     | -27.3    |
| time/              |          |
|    episodes        | 457700   |
|    fps             | 396      |
|    time_elapsed    | 20050    |
|    total_timesteps | 7943580  |
| train/             |          |
|    actor_loss      | 27.4     |
|    critic_loss     | 1.13     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.1     |
|    ep_rew_mean     | -27.8    |
| time/              |          |
|    episodes        | 459100   |
|    fps             | 396      |
|    time_elapsed    | 20111    |
|    total_timesteps | 7967544  |
| train/             |          |
|    actor_loss      | 28.4     |
|    critic_loss     | 0.804    |
|    ent_coef        | 0.0975   |
|    ent_coef_loss   | 0.142    |
|    learning_rate   | 0.0001   |
|    n_updates       | 663953   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.4     |
|    ep_rew_mean     | -28.6    |
| time/              |          |
|    episodes        | 459200   |
|    fps             | 396      |
|    time_elapsed    | 20115    |
|    total_timesteps | 7969284  |
| train/             |          |
|    actor_loss      | 27.7     |
|    critic_loss     | 1.25     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.6     |
|    ep_rew_mean     | -28.8    |
| time/              |          |
|    episodes        | 460600   |
|    fps             | 396      |
|    time_elapsed    | 20176    |
|    total_timesteps | 7993308  |
| train/             |          |
|    actor_loss      | 27.6     |
|    critic_loss     | 0.61     |
|    ent_coef        | 0.0975   |
|    ent_coef_loss   | -0.27    |
|    learning_rate   | 0.0001   |
|    n_updates       | 666100   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16.6     |
|    ep_rew_mean     | -27.4    |
| time/              |          |
|    episodes        | 460700   |
|    fps             | 396      |
|    time_elapsed    | 20180    |
|    total_timesteps | 7994964  |
| train/             |          |
|    actor_loss      | 27.6     |
|    critic_loss     | 1.06     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.4     |
|    ep_rew_mean     | -28.5    |
| time/              |          |
|    episodes        | 462100   |
|    fps             | 396      |
|    time_elapsed    | 20241    |
|    total_timesteps | 8018904  |
| train/             |          |
|    actor_loss      | 27.7     |
|    critic_loss     | 0.887    |
|    ent_coef        | 0.0974   |
|    ent_coef_loss   | 0.0506   |
|    learning_rate   | 0.0001   |
|    n_updates       | 668233   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16.6     |
|    ep_rew_mean     | -27.7    |
| time/              |          |
|    episodes        | 462200   |
|    fps             | 396      |
|    time_elapsed    | 20245    |
|    total_timesteps | 8020560  |
| train/             |          |
|    actor_loss      | 27.9     |
|    critic_loss     | 0.786    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.7     |
|    ep_rew_mean     | -28.9    |
| time/              |          |
|    episodes        | 463600   |
|    fps             | 396      |
|    time_elapsed    | 20305    |
|    total_timesteps | 8044320  |
| train/             |          |
|    actor_loss      | 27.5     |
|    critic_loss     | 1.19     |
|    ent_coef        | 0.0974   |
|    ent_coef_loss   | 0.153    |
|    learning_rate   | 0.0001   |
|    n_updates       | 670351   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.5     |
|    ep_rew_mean     | -29.3    |
| time/              |          |
|    episodes        | 463700   |
|    fps             | 396      |
|    time_elapsed    | 20310    |
|    total_timesteps | 8046096  |
| train/             |          |
|    actor_loss      | 28       |
|    critic_loss     | 1.04     |
|    ent_coef 

In [ ]:
model.save("trained_models/g1-reptile/sac")